In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
# basic librari
import numpy as np
import pandas as pd

# visulazation
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode

# SMOTE
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Deep Learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import os

#Remove warnings
import warnings
warnings.filterwarnings("ignore")

In [17]:
credit = pd.read_csv('/content/drive/MyDrive/Credit risk analysis/credit_risk.csv')

In [18]:
# Drop columns with missing values
credit.dropna(axis=1, inplace=True)

In [19]:
credit

,person_age,person_income,person_home_ownership,loan_intent,loan_grade,loan_amnt,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,PERSONAL,D,35000,1,0.59,Y,3
1,21,9600,OWN,EDUCATION,B,1000,0,0.10,N,2
2,25,9600,MORTGAGE,MEDICAL,C,5500,1,0.57,N,3
3,23,65500,RENT,MEDICAL,C,35000,1,0.53,N,2
4,24,54400,RENT,MEDICAL,C,35000,1,0.55,Y,4
...,...,...,...,...,...,...,...,...,...,...
32576,57,53000,MORTGAGE,PERSONAL,C,5800,0,0.11,N,30
32577,54,120000,MORTGAGE,PERSONAL,A,17625,0,0.15,N,19
32578,65,76000,RENT,HOMEIMPROVEMENT,B,35000,1,0.46,N,28
32579,56,150000,MORTGAGE,PERSONAL,B,15000,0,0.10,N,26


In [20]:
from sklearn.preprocessing import LabelEncoder

# Create LabelEncoder object
label_encoder = LabelEncoder()

# Apply label encoding to each categorical column
categorical_cols = ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']

for col in categorical_cols:
    credit[col] = label_encoder.fit_transform(credit[col])

# Check the updated DataFrame
credit.head()

,person_age,person_income,person_home_ownership,loan_intent,loan_grade,loan_amnt,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,3,4,3,35000,1,0.59,1,3
1,21,9600,2,1,1,1000,0,0.10,0,2
2,25,9600,0,3,2,5500,1,0.57,0,3
3,23,65500,3,3,2,35000,1,0.53,0,2
4,24,54400,3,3,2,35000,1,0.55,1,4


In [21]:
# Split data into features (X) and target variable (y)
X = credit.drop('loan_status', axis=1)
y = credit['loan_status']

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
# Define the neural network architecture
NNmodel = Sequential([
    Dense(64, input_shape=(X_train.shape[1],), activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),  # Additional hidden layer
    Dense(8, activation='relu'),   # Additional hidden layer
    Dense(1, activation='sigmoid')
])

# Compile the model
NNmodel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define TensorBoard callback with an absolute log directory path
log_dir = os.path.join(r"/content/drive/MyDrive/Credit risk analysis", "logs")  # Use 'r' before the string to treat it as a raw string
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Define other callbacks (EarlyStopping and ModelCheckpoint)
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss'),
    tf.keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/Credit risk analysis/best_model.keras', monitor='val_accuracy', save_best_only=True)
]

# Train the model with TensorBoard callback
history = NNmodel.fit(X_train, Y_train, epochs=50, batch_size=32,
                    validation_data=(X_test, Y_test),
                    callbacks=[tensorboard_callback] + callbacks)

# Evaluate the model on the test set
test_loss, test_accuracy = NNmodel.evaluate(X_test, Y_test)
print(f"Test Accuracy: {test_accuracy}")


Epoch 1/50
815/815 [==============================] - 8s 8ms/step - loss: 31.1839 - accuracy: 0.7361 - val_loss: 10.6712 - val_accuracy: 0.8168
Epoch 2/50
815/815 [==============================] - 4s 5ms/step - loss: 11.0492 - accuracy: 0.7377 - val_loss: 0.6483 - val_accuracy: 0.8054
Epoch 3/50
815/815 [==============================] - 3s 3ms/step - loss: 0.6693 - accuracy: 0.7810 - val_loss: 0.5016 - val_accuracy: 0.8146
Epoch 4/50
815/815 [==============================] - 3s 4ms/step - loss: 0.5342 - accuracy: 0.7959 - val_loss: 0.6719 - val_accuracy: 0.7915
Epoch 5/50
815/815 [==============================] - 3s 4ms/step - loss: 0.5112 - accuracy: 0.8088 - val_loss: 0.6039 - val_accuracy: 0.7884
Epoch 6/50
815/815 [==============================] - 4s 5ms/step - loss: 0.5035 - accuracy: 0.8151 - val_loss: 0.4809 - val_accuracy: 0.8160
Epoch 7/50
815/815 [==============================] - 3s 4ms/step - loss: 0.4807 - accuracy: 0.8207 - val_loss: 0.4825 - val_accuracy: 0.8154
Epo

In [24]:
new_model = tf.keras.models.load_model('/content/drive/MyDrive/Credit risk analysis/best_model.keras')

# Show the model architecture
new_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 64)                640       
                                                                 
 dense_11 (Dense)            (None, 32)                2080      
                                                                 
 dense_12 (Dense)            (None, 16)                528       
                                                                 
 dense_13 (Dense)            (None, 8)                 136       
                                                                 
 dense_14 (Dense)            (None, 1)                 9         
                                                                 
Total params: 3393 (13.25 KB)
Trainable params: 3393 (13.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
import tensorflow as tf

# Load the Keras model
model = tf.keras.models.load_model('/content/drive/MyDrive/Credit risk analysis/best_model.keras')

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to file
with open('/content/drive/MyDrive/Credit risk analysis/model.tflite', 'wb') as f:
    f.write(tflite_model)


In [28]:
import numpy as np
import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="/content/drive/MyDrive/Credit risk analysis/model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


In [29]:
def get_input_data():
    person_age = int(input("Enter person's age: "))
    person_income = int(input("Enter person's income: "))
    person_home_ownership = int(input("Enter person's home ownership (0 - Rented, 1 - Mortgage, 2 - Own, 3 - Other): "))
    loan_intent = int(input("Enter loan intent (1 - Debt Consolidation, 2 - Home Improvement, 3 - Medical Expenses, 4 - Other, 5 - Personal): "))
    loan_grade = int(input("Enter loan grade (1 - A, 2 - B, 3 - C, 4 - D, 5 - E, 6 - F, 7 - G): "))
    loan_amnt = int(input("Enter loan amount: "))
    loan_percent_income = float(input("Enter loan percent of income: "))
    cb_person_default_on_file = int(input("Enter person's default history (0 - No, 1 - Yes): "))
    cb_person_cred_hist_length = int(input("Enter person's credit history length (in years): "))

    # Prepare input data as a numpy array
    input_data = np.array([[person_age, person_income, person_home_ownership, loan_intent, loan_grade,
                            loan_amnt, loan_percent_income, cb_person_default_on_file, cb_person_cred_hist_length]],
                          dtype=np.float32)

    return input_data


In [30]:
def predict_loan_status(input_data):
    # Ensure the input shape matches the model's input tensor shape
    input_shape = input_details[0]['shape']
    input_data = np.reshape(input_data, input_shape)

    # Set input tensor
    interpreter.set_tensor(input_details[0]['index'], input_data)

    # Run inference
    interpreter.invoke()

    # Get output tensor
    output_data = interpreter.get_tensor(output_details[0]['index'])

    # Since it's a binary classification (assuming sigmoid output), round to get binary prediction
    binary_prediction = 1 if output_data[0][0] >= 0.5 else 0

    return binary_prediction


In [31]:
while True:
    # Get input data from user
    input_data = get_input_data()

    # Predict loan status
    prediction = predict_loan_status(input_data)

    # Print prediction
    print(f"Predicted loan status (0 - Not Defaulted, 1 - Defaulted): {prediction}")

    # Ask if user wants to continue
    continue_prompt = input("Do you want to predict again? (yes/no): ").lower()
    if continue_prompt != 'yes':
        break


Enter person's age: 22
Enter person's income: 59000
Enter person's home ownership (0 - Rented, 1 - Mortgage, 2 - Own, 3 - Other): 0
Enter loan intent (1 - Debt Consolidation, 2 - Home Improvement, 3 - Medical Expenses, 4 - Other, 5 - Personal): 5
Enter loan grade (1 - A, 2 - B, 3 - C, 4 - D, 5 - E, 6 - F, 7 - G): 4
Enter loan amount: 35000
Enter loan percent of income: 0.59
Enter person's default history (0 - No, 1 - Yes): 1
Enter person's credit history length (in years): 3
Predicted loan status (0 - Not Defaulted, 1 - Defaulted): 1
Do you want to predict again? (yes/no): no
